In [1]:
import numpy as np
import pandas as pd
import sys
from sklearn.metrics.pairwise import manhattan_distances

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indic

In [2]:
f=open('/Users/iamlira/Downloads/IOTest2.txt')
data=f.read().splitlines()
data_seq=[]
for tmp in data:
    d=tmp.split(' ')
    d.remove('')
    data_seq.append(d)

In [3]:
def pre_data(data):
    n=len(data)
    i=0
    while i<n:
        if(data[i][0]=='connect'):
            if(data[i-1][0]=='move_start'):
                del(data[i-1])
                i=i-1
                n=len(data)
                continue
            if(data[i+1][0]=='move_end'):
                del(data[i+1])
                n=len(data)
                continue
        if(data[i][0]=='assign'):
            if(data[i-1][0]=='move_end' and data[i-2][0]=='move_end' and data[i-3][0]=='move_start'):
                del(data[i-1])
                del(data[i-2])
                del(data[i-3])
                i=i-2
                n=len(data)
                continue
        n=len(data)
        i=i+1
    
pre_data(data_seq)
for s in data_seq:
    print(s)

['AddPou', '2018/9/4', '21:46:19']
['choose_start', '2018/9/4', '21:46:45']
['choose_end', '2018/9/4', '21:46:46']
['choose_start', '2018/9/4', '21:47:05']
['choose_end', '2018/9/4', '21:47:07']
['choose_start', '2018/9/4', '21:47:11']
['choose_end', '2018/9/4', '21:47:12']
['choose_start', '2018/9/4', '21:47:16']
['choose_end', '2018/9/4', '21:47:17']
['choose_start', '2018/9/4', '21:47:23']
['choose_end', '2018/9/4', '21:47:24']
['choose_start', '2018/9/4', '21:47:28']
['choose_end', '2018/9/4', '21:47:29']
['choose_start', '2018/9/4', '21:47:35']
['choose_end', '2018/9/4', '21:47:36']
['choose_start', '2018/9/4', '21:47:43']
['choose_end', '2018/9/4', '21:47:44']
['choose_start', '2018/9/4', '21:47:52']
['choose_end', '2018/9/4', '21:47:54']
['connect', '2018/9/4', '21:48:22']
['connect', '2018/9/4', '21:48:23']
['connect', '2018/9/4', '21:48:24']
['connect', '2018/9/4', '21:48:26']
['connect', '2018/9/4', '21:48:27']
['connect', '2018/9/4', '21:48:28']
['connect', '2018/9/4', '21:4

In [4]:
def lcs(a,b):
  lena=len(a)
  lenb=len(b)
  c=[[0 for i in range(lenb+1)] for j in range(lena+1)]
  flag=[[0 for i in range(lenb+1)] for j in range(lena+1)]
  for i in range(lena):
    for j in range(lenb):
      if a[i]==b[j]:
        c[i+1][j+1]=c[i][j]+1
        flag[i+1][j+1]='ok'
      elif c[i+1][j]>c[i][j+1]:
        c[i+1][j+1]=c[i+1][j]
        flag[i+1][j+1]='left'
      else:
        c[i+1][j+1]=c[i][j+1]
        flag[i+1][j+1]='up'
  return c,flag

def printLcs(flag,a,i,j):
  if i==0 or j==0:
    return
  if flag[i][j]=='ok':
    printLcs(flag,a,i-1,j-1)
    print(a[i-1],end='')
  elif flag[i][j]=='left':
    printLcs(flag,a,i,j-1)
  else:
    printLcs(flag,a,i-1,j)
    
a='ABCBDAB'
b='BDCABA'
c,flag=lcs(a,b)
#for i in c:
#  print(i)
#print('')
#for j in flag:
#  print(j)
print('')
printLcs(flag,a,len(a),len(b))
print('')


BCBA


In [5]:
def dtw(X,Y):
     M=[[manhattan_distances(X[i],Y[i]) for i in range(len(X))] for j in range(len(Y))]
     l1=len(X)
     l2=len(Y) 
     D=[[0 for i in range(l1+1)] for i in range(l2+1)]
     D[0][0]=0 
     for i in range(1,l1+1):
         D[0][i]=sys.maxsize
     for j in range(1,l2+1):
         D[j][0]=sys.maxsize
     for j in range(1,l2+1):
         for i in range(1,l1+1):
             D[j][i]=M[j-1][i-1]+min(D[j-1][i],D[j][i-1],D[j-1][i-1]+M[j-1][i-1])
     print(D[-1][-1])

In [6]:
X=[1,2,3,4]
Y=[1,2,7,4,5]
dtw(X,Y)

[[4.]]


In [7]:
def edit_distance(str1, str2):
    matrix = [[i+j for j in range(len(str2) + 1)] for i in range(len(str1) + 1)]
    for i in range(1,len(str1)+1):
        for j in range(1,len(str2)+1):
            if str1[i-1] == str2[j-1]:
                d = 0
            else:
                d = 1
            matrix[i][j] = min(matrix[i-1][j]+1,matrix[i][j-1]+1,matrix[i-1][j-1]+d)
    return matrix[len(str1)][len(str2)]

print(edit_distance('1234','12745'))
print(manhattan_distances('1234','12745'))

2
[[11511.]]


In [8]:
def k_gram(seq,k):
    kgram=set()
    n=len(seq)
    for i in range(n-k+1):
        kgram.add(tuple(seq[i:i+k]))
    return kgram

def Dk(gram_1,gram_2):
    intersection=gram_1.intersection(gram_2)
    union=gram_1.union(gram_2)
    return 1-(len(intersection)/len(union))

def my_algo(x,y,k):
    x_gram=k_gram(x,k)
    y_gram=k_gram(y,k)
    return Dk(x_gram,y_gram)

a=[2,3,4]
b=[1,2,3,4]
print(my_algo(a,b,3))

0.5
